In [1]:
import sys
from pathlib import Path
import numpy as np
from scipy.ndimage import center_of_mass
#import warnings
#warnings.filterwarnings("error")
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np

PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.utilities.algorithm import umeyama

/home/eddyod/programming/preprocessing-pipeline/src


In [2]:
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))

def plot_point_sets_3d(point_sets):
    df = pd.DataFrame()
    for data, label in point_sets:
        df_cur = pd.DataFrame(data.T, columns=['x', 'y', 'z'])
        df_cur['label'] = label
        #df = df.concat(df_cur, ignore_index=True)
        df = pd.concat([df, df_cur], axis= 0)
    
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='label')
    return fig

In [3]:
animal = 'Atlas'
brain = BrainStructureManager(animal)
brain.fixed_brain = BrainStructureManager('Allen')

In [4]:
moving_coms = brain.get_coms(annotator_id=1)
#del moving_coms['Sp5C_L']
#del moving_coms['RtTg']
fixed_coms = brain.fixed_brain.get_coms(annotator_id=1)
common_keys = fixed_coms.keys() & moving_coms.keys()
brain_regions = sorted(moving_coms.keys())
common_keys = ['IC','PBG_L','PBG_R','3N_L','3N_R','4N_L','4N_R','SNR_L','SNR_R','VLL_L','VLL_R']
fixed_points = np.array([fixed_coms[s] for s in brain_regions if s in common_keys])
moving_points = np.array([moving_coms[s] for s in brain_regions if s in common_keys])

In [5]:
fixed_point_dict = {s:fixed_coms[s] for s in brain_regions if s in common_keys}
moving_point_dict = {s:moving_coms[s] for s in brain_regions if s in common_keys}

In [6]:
fixed_point_dict

{'3N_L': [9103.0, 3794.25, 5500.0],
 '3N_R': [9103.0, 3794.25, 5875.0],
 '4N_L': [9587.75, 3776.25, 5450.0],
 '4N_R': [9587.75, 3776.25, 5950.0],
 'IC': [10402.5, 2332.25, 5692.0],
 'PBG_L': [9401.75, 3845.25, 3575.0],
 'PBG_R': [9401.75, 3845.25, 7825.0],
 'SNR_L': [8440.53, 5167.26, 4250.0],
 'SNR_R': [8440.53, 5167.26, 7225.0],
 'VLL_L': [9464.75, 5174.75, 4200.0],
 'VLL_R': [9464.75, 5174.75, 7400.0]}

In [7]:
distances = []
for structure in common_keys:
    (x,y,z) = fixed_point_dict[structure]
    fixed_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    d = calculate_distance(fixed_point, moving_point)
    distances.append(d)
    moving_point = np.round(moving_point/25)
    print(f'{structure} COM={moving_point} distance={round(d,2)}')

IC COM=[418.  88. 226.] distance=147.4
PBG_L COM=[380. 146. 144.] distance=218.28
PBG_R COM=[378. 146. 315.] distance=211.59
3N_L COM=[361. 154. 225.] distance=148.61
3N_R COM=[362. 155. 236.] distance=87.61
4N_L COM=[383. 154. 217.] distance=77.27
4N_R COM=[382. 155. 240.] distance=104.77
SNR_L COM=[342. 197. 169.] distance=265.15
SNR_R COM=[339. 194. 296.] distance=350.74
VLL_L COM=[381. 184. 150.] distance=720.4
VLL_R COM=[379. 187. 307.] distance=570.64


In [ ]:
#IC COM=[420. 101. 231.] distance=235.02
#PBG_L COM=[380. 150. 151.] distance=239.76
#PBG_R COM=[375. 153. 311.] distance=57.22
#3N_L COM=[361. 157. 226.] distance=220.24
#3N_R COM=[361. 158. 237.] distance=185.78
#4N_L COM=[381. 159. 219.] distance=203.86
#4N_R COM=[380. 160. 241.] distance=244.41
#SNR_L COM=[339. 196. 173.] distance=287.68
#SNR_R COM=[333. 195. 291.] distance=309.48
#VLL_L COM=[376. 189. 157.] distance=522.31
#VLL_R COM=[370. 195. 301.] distance=386.75

In [ ]:
print(fixed_points.shape)
print('Mean')
print(np.mean(fixed_points, axis=0))
print(np.mean(moving_points, axis=0))
print('Min')
print(np.min(fixed_points, axis=0))
print(np.min(moving_points, axis=0))
print('Max')
print(np.max(fixed_points, axis=0))
print(np.max(moving_points, axis=0))
#fixed_points - moving_points

In [ ]:
plot_point_sets_3d([
    (moving_points.T, 'unaligned moving centers'),
    (fixed_points.T, 'Allen centers')
])

In [ ]:
r, t = umeyama(moving_points.T, fixed_points.T)

In [ ]:
t

In [ ]:
reg_points = r @ moving_points.T + t

In [ ]:
print('Mean')
print(np.mean(fixed_points, axis=0))
print(np.mean(reg_points, axis=1))
print('Min')
print(np.min(fixed_points, axis=0))
print(np.min(reg_points, axis=1))
print('Max')
print(np.max(fixed_points, axis=0))
print(np.max(reg_points, axis=1))

In [ ]:
reg_point_dict = {s:moving_coms[s] for s in brain_regions if s in common_keys}

In [ ]:
distances = []
for structure in common_keys:
    (x,y,z) = fixed_point_dict[structure]
    fixed_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, r, t)
    d = calculate_distance(fixed_point, reg_point)
    distances.append(d)
    reg_point = np.round(reg_point / 25)
    print(f'{structure} COM={reg_point} distance={round(d,2)}')

In [ ]:
len(distances), round(np.mean(distances)), round(min(distances)), round(max(distances)), round(np.sum(distances))

In [ ]:
# MD594 (33, 436, 111, 2838, 14391)
# MD589 (33, 429, 107, 2922, 14146)
# MD589 (31, 276, 53, 818, 8554) # with no RtTG and SPc
# MD589 (33, 276, 77, 529, 9098)

In [ ]:
plot_point_sets_3d([
    (reg_points, 'registered moving centers'),
    (fixed_points.T, 'Allen centers')    
])